<a href="https://colab.research.google.com/github/iampaapa/for-peprah/blob/main/Peprah.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np

def cost_function(P, a, b, c, S, U, SP):
    cost = 0
    revenue = 0

    for i in range(len(P)):
        for t in range(len(P[i])):
            if U[i][t] == 1:
                cost += (a[i] + b[i] * P[i][t] + c[i] * P[i][t]**2 + S) * U[i][t]
                revenue += P[i][t] * SP[t % 10] * U[i][t]  # Corrected here to use SP[t % len(SP)]

    return cost, revenue

def gwo_optimization(P_min, P_max, P_demand, a, b, c, S, SP, U, max_iterations):
    num_units = len(P_min)
    num_times = len(P_demand)
    alpha_pos = np.zeros((num_units, num_times))
    beta_pos = np.zeros((num_units, num_times))
    delta_pos = np.zeros((num_units, num_times))
    alpha_score = float('inf')
    beta_score = float('inf')
    delta_score = float('inf')

    # Initialize positions
    positions = np.zeros((num_units, num_times))
    for i in range(num_units):
        for t in range(num_times):
            positions[i][t] = np.random.uniform(P_min[i], P_max[i])

    # GWO main loop
    iteration = 0
    while iteration < max_iterations:
        for i in range(num_units):
            for t in range(num_times):
                # Update alpha, beta, and delta positions
                if cost_function(positions, a, b, c, S, U, SP)[0] < alpha_score:
                    alpha_pos = positions.copy()
                    alpha_score = cost_function(positions, a, b, c, S, U, SP)[0]

                if (cost_function(positions, a, b, c, S, U, SP)[0] > alpha_score) and (cost_function(positions, a, b, c, S, U, SP)[0] < beta_score):
                    beta_pos = positions.copy()
                    beta_score = cost_function(positions, a, b, c, S, U, SP)[0]

                if (cost_function(positions, a, b, c, S, U, SP)[0] > alpha_score) and (cost_function(positions, a, b, c, S, U, SP)[0] > beta_score) and (cost_function(positions, a, b, c, S, U, SP)[0] < delta_score):
                    delta_pos = positions.copy()
                    delta_score = cost_function(positions, a, b, c, S, U, SP)[0]

                # Update positions of wolves
                A1 = 2 * (1 - iteration / max_iterations)
                C1 = 2 * np.random.random()
                D_alpha = np.abs(C1 * alpha_pos[i][t] - positions[i][t])
                X1 = alpha_pos[i][t] - A1 * D_alpha

                A2 = 2 * (1 - iteration / max_iterations)
                C2 = 2 * np.random.random()
                D_beta = np.abs(C2 * beta_pos[i][t] - positions[i][t])
                X2 = beta_pos[i][t] - A2 * D_beta

                A3 = 2 * (1 - iteration / max_iterations)
                C3 = 2 * np.random.random()
                D_delta = np.abs(C3 * delta_pos[i][t] - positions[i][t])
                X3 = delta_pos[i][t] - A3 * D_delta

                positions[i][t] = (X1 + X2 + X3) / 3

        iteration += 1

    return alpha_pos

if __name__ == "__main__":
    # Example usage
    P_min = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
    P_max = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
    P_demand = [700, 750, 850, 950, 1000, 1100, 1150, 1200, 1300, 1400, 1450, 1500, 1400, 1300, 1200, 1050, 1000, 1100, 1200, 1400, 1300, 1100, 900, 800]
    a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    b = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    c = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]
    S = 5000
    SP = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, ...]

    # Binary decision for unit on/off at each time (24 hours)
    U = np.random.randint(0, 2, size=(len(P_min), len(P_demand)))

    # GWO optimization
    max_iterations = 100
    optimized_power = gwo_optimization(P_min, P_max, P_demand, a, b, c, S, SP, U, max_iterations)

    print("Optimized power production:")
    print(optimized_power)

Optimized power production:
[[ -99.2538612    -5.65021271  -31.67493598  -67.9755832   -18.40893759
   -46.47568297  -61.94093584  -12.30480305   -5.89121024  -12.11085904
   -47.13258021   -7.04313267  -38.66322934  -29.82589184  -79.99850399
   -14.76655689  -58.56206394  -22.24871214  -42.73539032  -43.90536857
   -20.40263802  -10.13070138  -72.6542317   -19.78940105]
 [ -16.37162848 -167.61016114  -17.37430954 -168.41722025  -94.21675408
   -86.78519466  -25.06848045 -120.55861952  -59.91249097 -105.97049377
   -77.43824149  -28.48045727 -132.06006855  -27.06216356   -7.52267093
  -117.43634797  -82.42390717  -66.42449855  -27.37077207 -131.28291374
  -102.42328442 -127.85921073  -15.26185879  -41.25352516]
 [ -67.05264736 -143.77915617 -113.95731081 -173.87099352 -262.48500492
  -136.8508432   -46.25929712  -64.27528645 -188.4687392  -194.57123649
   -72.88262733  -10.13519248  -28.2307031   -73.20384504  -37.2702597
   -80.24022707 -137.31141991  -96.64263258 -169.21632651  -47.